In [ ]:
! pip install transformers datasets
! pip install rouge-score nltk
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt install git-lfs
#!git config --global user.email "rahulverma207788@gmail.com"
#!git config --global user.name "rahul77"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
import transformers
print(transformers.__version__)

4.23.1


In [ ]:
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd
from datasets import load_dataset, dataset_dict, load_metric
from datasets.dataset_dict import DatasetDict
import datasets
import random
from IPython.display import display, HTML
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import requests

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
metric = load_metric("rouge")

### Loding the dataset

In [ ]:
data = pd.read_csv("/content/the_hindu.csv")
data.drop(data.filter(regex="Unnamed"),axis=1, inplace=True)
data.head(3)

,Document,Original Summary,Corrected Summary
0,"Tripti Tandon, advocate for the Durbar Mahila ...","Supreme Court order on May 19 ""does not recogn...","The May 19 Supreme Court order ""does not recog..."
1,India's top oil and gas producer ONGC on Thurs...,India's top oil and gas producer ONGC says it ...,India's top oil and gas producer ONGC says it ...
2,Pakistan's National Assembly on May 26 passed ...,The Elections (Amendment) Bill 2022 passed wit...,The lower house of Pakistan's National Assembl...


In [ ]:
data = data.drop(['Original Summary'], axis=1)

In [ ]:
data['id'] = data[['Document', 'Corrected Summary']].sum(axis=1).map(hash)

In [ ]:
data.head(3)

,Document,Corrected Summary,id
0,"Tripti Tandon, advocate for the Durbar Mahila ...","The May 19 Supreme Court order ""does not recog...",-4764630109383279921
1,India's top oil and gas producer ONGC on Thurs...,India's top oil and gas producer ONGC says it ...,648404895034202308
2,Pakistan's National Assembly on May 26 passed ...,The lower house of Pakistan's National Assembl...,-2257659160371720420


In [ ]:
data.to_csv('the_hindu_1.csv')

In [ ]:
dataset = load_dataset('csv', data_files='/content/the_hindu_1.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-89a0abe48b7e4198/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data1 = data.copy()

In [ ]:
data1.head(5)

,Document,Corrected Summary,id
0,"Tripti Tandon, advocate for the Durbar Mahila ...","The May 19 Supreme Court order ""does not recog...",-4764630109383279921
1,India's top oil and gas producer ONGC on Thurs...,India's top oil and gas producer ONGC says it ...,648404895034202308
2,Pakistan's National Assembly on May 26 passed ...,The lower house of Pakistan's National Assembl...,-2257659160371720420
3,Billionaire Elon Musk was sued by Twitter Inc...,Elon Musk has been sued by Twitter Inc. invest...,-300396398898335664
4,Prime Minister Narendra Modi on Thursday dedi...,Prime Minister Narendra Modi laid the foundati...,5634013281684312872


In [ ]:
summaries, longtext = pd.DataFrame(), pd.DataFrame()

In [ ]:
summaries['short'] = data1['Corrected Summary']
longtext['long'] = data1['Document']

In [ ]:
(summaries.shape,longtext.shape)

((179, 1), (179, 1))

In [ ]:
summaries.head(2)

,short
0,"The May 19 Supreme Court order ""does not recog..."
1,India's top oil and gas producer ONGC says it ...


In [ ]:
longtext.head(2)

,long
0,"Tripti Tandon, advocate for the Durbar Mahila ..."
1,India's top oil and gas producer ONGC on Thurs...


In [ ]:
def max_length(shorts, longs, prct):
    length_longs = list(len(d.split()) for d in longs)
    length_shorts = list(len(d.split()) for d in shorts)
    print('percentile {} of length of news: {}'.format(prct,np.percentile(length_longs, prct)))
    print('longest sentence: ', max(length_longs))
    print()
    print('percentile {} of length of summaries: {}'.format(prct,np.percentile(length_shorts, prct)))
    print('longest sentence: ', max(length_shorts))
    print()
    return int(np.percentile(length_longs, prct)),int(np.percentile(length_shorts, prct))
max_len_news, max_len_summary= max_length(summaries['short'].to_list(), longtext['long'].to_list(), 90)
print('max-length longtext chosen for training: ', max_len_news)
print('max-length summaries chosen for training: ', max_len_summary)

percentile 90 of length of news: 674.0000000000001
longest sentence:  1123

percentile 90 of length of summaries: 104.0
longest sentence:  120

max-length longtext chosen for training:  674
max-length summaries chosen for training:  104


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Document', 'Corrected Summary', 'id'],
        num_rows: 179
    })
})

In [ ]:
dataset = dataset.remove_columns('Unnamed: 0')

### Splitting the dataset

In [ ]:
train_devtest = dataset['train'].train_test_split(shuffle = True, seed = 200, test_size=0.3)
posts_dev_test = train_devtest['test'].train_test_split(shuffle = True, seed = 200, test_size=0.50)
raw_dataset = DatasetDict({
    'train': train_devtest['train'],
    'test': posts_dev_test['test'],
    'validation': posts_dev_test['train']})

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Document', 'Corrected Summary', 'id'],
        num_rows: 125
    })
    test: Dataset({
        features: ['Document', 'Corrected Summary', 'id'],
        num_rows: 27
    })
    validation: Dataset({
        features: ['Document', 'Corrected Summary', 'id'],
        num_rows: 27
    })
})

In [ ]:
raw_dataset["train"][0]

{'Document': 'GUWAHATI The migration of border villagers has added to the challenges the Indian Army has been facing along the Line of Actual Control (LAC) in trying to match the rapid enhancement of infrastructure and operational capabilities by the People\'s Liberation Army (PLA) of China. Lieutenant General Rana Pratap Kalita, chief of the Army\'s Eastern Command, said this in Guwahati on Monday, while asserting that the partial withdrawal of the Armed Forces (Special Powers) Act from three northeastern States has aided the redeployment of troops on the northern borders of the eastern sector and focus more on China. "Migration of people from near the LAC in Arunachal Pradesh has led to shortage of people to work in those areas. The outmigration to the hinterland has been compounded by the unwillingness of people from outside to work in those difficult areas," he said. Hundreds of villagers have moved to towns in the more accessible areas in search of greener pastures over the years.

### Showing the elements

In [ ]:
def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_dataset["train"])

,Document,Corrected Summary,id
0,"The government on Thursday opted to leave interest rates on small savings instruments like the Public Provident Fund unchanged for April to June 2022, maintaining the status quo for the eighth successive quarter. The Reserve Bank of India had indicated the need to pare interest rates on these schemes between 0.09 and 1.18 percentage points in the coming quarter. However, the resurgence of inflation beyond 6% in the last two months and the upswing in oil prices may have weighed against taking an unpopular step, especially after the Employees' Provident Fund rate for 2021-22 was reduced from 8.5% to 8.1% earlier this month, the lowest in over 42 years. The rates on various small savings schemes were last reduced in the range of 0.5 and 1.4 percentage points in April 2020, and an announcement of further cuts ranging from 0.4% to 1.1% on March 31 , 2022, was revoked by April 1. The quarterly interest rate paid on one year term deposits remains 5.5%, Senior Citizen Savings' Scheme will continue to earn 7.4%, while the Sukanya Samriddhi Account Scheme's return is 7.6%. The PPF return will remain at 7.1%. ICRA chief economist Aditi Nayar said small savings rates may be increased in the latter half of the year following expected rate hikes by the central bank through August and October. ""With the rise in yields on government securities over the last three months, as well as the inching up of deposit rates of banks, we had foreseen a small probability of the small savings rates being revised upwards for the coming quarter,"" she noted.","The government on April 1 opted to leave interest rates on small savings instruments like the Public Provident Fund unchanged for April to June 2022. The quarterly interest rate paid on one year term deposits remains 5.5%, Senior Citizen Savings' Scheme will continue to earn 7.4%, while the Sukanya Samriddhi Account Scheme's return is 7.6%. The PPF return will remain at 7.1%. The RBI had indicated the need to pare interest rates on these schemes in the coming quarter, but the resurgence of inflation in the last two months and the upswing in oil prices may have weighed against taking an unpopular step.",7307496497013463363
1,"Chinese leader Xi Jinping on March 31 issued strong backing for Afghanistan at a regional conference, while making no mention of human rights abuses by the country's Taliban leaders. Xi pledged China's support in a message to a gathering of representatives from Afghanistan, China, Russia, Pakistan, Iran, Tajikistan, Turkmenistan and Uzbekistan in a central Chinese city that spotlights Beijing's aspirations to play a leading role in Afghanistan following the withdrawal of U.S. forces last August. A ""peaceful, stable, developed and prosperous Afghanistan"" is what Afghans aspire to, which ""serves the common interests of regional countries and the international community,"" Xi said. ""China has all along respected Afghanistan's sovereignty, independence and territorial integrity, and is committed to supporting Afghanistan's peaceful and stable development,"" Xi said in his message to the gathering in Tunxi, a centre of the tourism industry in Anhui province. Xi gave no specifics, although China has already shipped emergency aid to Afghanistan and is seeking to develop copper mining there. China follows what it calls a strict policy of ""non-intervention"" in other countries' internal affairs, including opposing those staged for humanitarian purposes unless sanctioned by the United Nations. Despite that, Beijing is frequently accused of meddling to further its own domestic and international interests. Special envoys for Afghanistan from China, the United States and Russia, a group known as the ""Extended Troika,"" were also meeting concurrently in Tunxi. Although it has yet to recognise the Taliban government, China has moved quickly to shore up its ties with the radical Islamic group. A month before the Taliban took power, Foreign Minister Wang Yi 

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

### Loding the tokenizer and Preprocessing the data

In [ ]:
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 19, 3, 9, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
max_input_length = 800
max_target_length = 105
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["Corrected Summary"], max_length=max_target_length, truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_dataset["train"][:1])

{'input_ids': [[21603, 10, 3, 13961, 12054, 566, 21580, 37, 11661, 13, 4947, 3, 31101, 65, 974, 12, 8, 2428, 8, 2557, 6788, 65, 118, 5008, 590, 8, 4919, 13, 24193, 4330, 41, 4569, 254, 61, 16, 1119, 12, 1588, 8, 3607, 15220, 13, 3620, 11, 7763, 5644, 57, 8, 2449, 31, 7, 12774, 257, 6788, 41, 5329, 188, 61, 13, 1473, 5, 30943, 2146, 15039, 9, 6110, 8873, 5740, 155, 9, 6, 5752, 13, 8, 6788, 31, 7, 7345, 13901, 6, 243, 48, 16, 2846, 17771, 144, 23, 30, 2089, 6, 298, 15376, 53, 24, 8, 11807, 11905, 13, 8, 18251, 26, 5205, 7, 41, 7727, 23, 138, 2621, 7, 61, 1983, 45, 386, 25806, 49, 29, 1323, 65, 3, 5385, 26, 8, 1131, 15, 7379, 63, 297, 13, 12673, 30, 8, 8390, 15094, 13, 8, 13350, 2393, 11, 992, 72, 30, 1473, 5, 96, 329, 23, 122, 2661, 13, 151, 45, 1084, 8, 5292, 254, 16, 71, 4312, 9, 12654, 22660, 65, 2237, 12, 16316, 13, 151, 12, 161, 16, 273, 844, 5, 37, 91, 10673, 2661, 12, 8, 7841, 40, 232, 65, 118, 12771, 15, 26, 57, 8, 28963, 655, 13, 151, 45, 1067, 12, 161, 16, 273, 1256, 844, 976, 

In [ ]:
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### Loading the pretrained model

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, output_attentions=True)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


### Defining the parameters of the model

In [ ]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 50
model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-thehindu1"

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
tokenized_datasets["train"]

Dataset({
    features: ['Document', 'Corrected Summary', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 125
})

In [ ]:
show_random_elements(tokenized_datasets["train"])

### Creating the dataset which is ready to pass model

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from transformers import AdamWeightDecay
import tensorflow as tf
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result

### Training the model

In [ ]:
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard
tensorboard_callback = TensorBoard(log_dir="./summarization_model_save1/logs")
push_to_hub_callback = PushToHubCallback(
    output_dir="./summarization_model_save1",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)
metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True
)
callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]
model.fit(
    train_dataset, validation_data=validation_dataset, epochs=50, callbacks=callbacks
)

Cloning https://huggingface.co/rahul77/t5-small-finetuned-thehindu1 into local empty directory.


Epoch 1/50
 6/15 [===========>..................] - ETA: 4s - loss: 1.3226

15/15 [==============================] - 83s 4s/step - loss: 1.2252 - val_loss: 0.9927 - rouge1: 25.8031 - rouge2: 17.7261 - rougeL: 23.4483 - rougeLsum: 25.0648 - gen_len: 19.0000
Epoch 2/50
15/15 [==============================] - 10s 694ms/step - loss: 1.0509 - val_loss: 0.9137 - rouge1: 28.0482 - rouge2: 20.6823 - rougeL: 25.5396 - rougeLsum: 27.0125 - gen_len: 19.0000
Epoch 3/50
15/15 [==============================] - 10s 709ms/step - loss: 0.9961 - val_loss: 0.8638 - rouge1: 28.2964 - rouge2: 22.1783 - rougeL: 26.4157 - rougeLsum: 27.4368 - gen_len: 19.0000
Epoch 4/50
15/15 [==============================] - 11s 743ms/step - loss: 0.9266 - val_loss: 0.8321 - rouge1: 27.7054 - rouge2: 21.8853 - rougeL: 26.0306 - rougeLsum: 26.9068 - gen_len: 19.0000
Epoch 5/50
15/15 [==============================] - 11s 724ms/step - loss: 0.8851 - val_loss: 0.8117 - rouge1: 28.3740 - rouge2: 22.8198 - rougeL: 26.8479 - rougeLsum: 27.5047 - gen_len: 19.0000
Epoch 6/50
15/15 [=====================

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.34k/231M [00:00<?, ?B/s]

Upload file logs/train/events.out.tfevents.1666406257.6d528a7304c2.1548.0.v2:   0%|          | 3.34k/2.77M [00…

Upload file logs/validation/events.out.tfevents.1666406287.6d528a7304c2.1548.1.v2:  44%|####3     | 3.34k/7.65…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/rahul77/t5-small-finetuned-thehindu1
   4d73910..d70eaf9  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/rahul77/t5-small-finetuned-thehindu1
   4d73910..d70eaf9  main -> main



### Loading the model and predicting the results

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'rahul77/t5-small-finetuned-thehindu1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model1 = T5ForConditionalGeneration.from_pretrained(model_name, output_attentions=True, from_tf = True)

All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.


In [ ]:
def get_text_from_link(link):
    rsp = requests.get(link)
    if rsp.status_code != 200:
        return None
    soup = BeautifulSoup(rsp.text, features="lxml")
    all_p = soup.find_all("p")
    text = []
    for p in all_p:
        p_text = p.get_text()
        if (
            p.attrs != {} is not None
            or p_text.find("Photo Credit") != -1
            or p_text.find("Printable version") != -1
        ):
            continue
        p_text = (
            p_text.replace("‘", "'")
            .replace("’", "'")
            .replace("“", '"')
            .replace("”", '"')
            .replace(r"\'", "'")
        )
        text.append(p_text)
    text = " ".join(text)
    return text

In [ ]:
document1 = get_text_from_link("https://www.thehindu.com/business/sebi-amends-rules-governing-collective-investment-schemes/article65271602.ece")

In [ ]:
if 't5' in model_name: 
    document1 = "summarize: " + document1
tokenized = tokenizer([document1], return_tensors='np')
out = model.generate(**tokenized, max_length=128)

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

<pad> Securities and Exchange Board of India approved an amendment to the SEBI (Collective Investment Schemes) Regulations, 1999 to strengthen the regulatory framework for collective investment schemes (CIS) in line with mutual fund regulations to remove regulatory arbitrage. There will be a mandatory investment of CIMC and its designated employees in the Collective Investment Schemes (CIS) to align their interest with that of the CIS. There will be mandatory requirement of minimum number of investors, maximum holding of a single investor, and minimum subscription amount</s>


In [ ]:
document2 = get_text_from_link("https://www.thehindu.com/business/Industry/musk-sued-by-twitter-investors-for-delayed-disclosure-of-stake/article65464584.ece")

In [ ]:
if 't5' in model_name: 
    document2 = "summarize: " + document2
tokenized = tokenizer([document2], return_tensors='np')
out2 = model.generate(**tokenized, max_length=128)

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out2[0]))

<pad> Elon Musk sued Twitter Inc investors for delayed disclosure of his stake in the social media company. The investors said Mr. Musk saved himself $156 million by failing to disclose that he had purchased more than 5% of Twitter by March 14. He continued to buy stock after that, and ultimately disclosed in early April that he owned 9.2% of the company. The suit comes after Mr. Musk, the owner of electric carmaker Tesla Inc, said he planned to take over Twitter for $44 billion.</s>


### Using XLA and predicting results

In [ ]:
import tensorflow as tf
@tf.function(jit_compile=True)
def generate(inputs):
    return model.generate(**inputs, max_length=128)
tokenized_data = tokenizer([document], return_tensors="np", pad_to_multiple_of=128)
out = generate(tokenized_data)

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

<pad> The Securities and Exchange Board of India approved an amendment to the SEBI Regulations, 1999 to strengthen the regulatory framework for collective investment schemes. The board also approved a proposal to amend the Securities and Exchange Board of India (SEBI) Regulations, 2015, for simplification of procedure for transmission of securities. The existing threshold limit for simplified documents has been revised from <unk>2 lakh to <unk>5 lakh for securities held in physical mode per listed issuer, and <unk>5 lakh to <unk> 15 lakh for securities held in dematerialised mode for</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

The Securities and Exchange Board of India approved an amendment to the SEBI Regulations, 1999 to strengthen the regulatory framework for collective investment schemes. The board also approved a proposal to amend the Securities and Exchange Board of India (SEBI) Regulations, 2015, for simplification of procedure for transmission of securities. The existing threshold limit for simplified documents has been revised from 2 lakh to 5 lakh for securities held in physical mode per listed issuer, and 5 lakh to  15 lakh for securities held in dematerialised mode for


In [ ]:
from transformers import pipeline
summarizer = pipeline('text2text-generation', model_name, framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at rahul77/t5-small-finetuned-thehindu1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
summarizer(document, max_length=128)

[{'generated_text': 'The Securities and Exchange Board of India approved an amendment to the SEBI Regulations, 1999 to strengthen the regulatory framework for collective investment schemes. The board also approved a proposal to amend the Securities and Exchange Board of India (SEBI) Regulations, 2015, for simplification of procedure for transmission of securities. The existing threshold limit for simplified documents has been revised from 2 lakh to 5 lakh for securities held in physical mode per listed issuer, and 5 lakh to  15 lakh for securities held in dematerialised mode for'}]

### **Visualization of attention of each layer**

In [ ]:
!pip install bertviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [175]:
from bertviz import model_view
from bertviz.neuron_view import show
from bertviz import head_view

In [176]:
encoder_input_ids = tokenizer("The House Budget Committee voted Saturday to pass a $3.5 trillion spending bill", return_tensors="pt", add_special_tokens=True).input_ids

In [177]:
decoder_input_ids = tokenizer("The House Budget Committee passed a spending bill.", return_tensors="pt", add_special_tokens=True).input_ids

In [178]:
outputs1 = model1(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)

In [179]:
encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0])


In [180]:
model_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens= encoder_text,
    decoder_tokens=decoder_text
)

<IPython.core.display.Javascript object>

In [181]:
head_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens= encoder_text,
    decoder_tokens=decoder_text
)

<IPython.core.display.Javascript object>